In [0]:
import smtplib
import base64
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from io import BytesIO

In [0]:
def fetch_csv_attachment(table_name: str):
    df = spark.read.table(table_name)
    if df.count() == 0:
        return None
    
    pdf = df.toPandas()
    buffer = BytesIO()
    pdf.to_csv(buffer, index=False)
    buffer.seek(0)
    
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(buffer.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', f'attachment; filename="{table_name.replace(".", "_")}.csv"')
    
    return part

In [0]:
def send_gmail_email(from_email, to_email, app_password, tables: list):
    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = "[Databricks Alert] Failed & Quarantine Records"

    msg.attach(MIMEText("""
    <p>Hi Team,</p>
    <p>Attached are the failed and quarantined records from the pipeline run.</p>
    <p>Regards,<br>Databricks</p>
    """, 'html'))

    # Attach CSVs
    for table in tables:
        part = fetch_csv_attachment(table)
        if part:
            msg.attach(part)

    if len(msg.get_payload()) <= 1:
        print("✅ No failed or quarantined records to send.")
        return

    # Send email via Gmail SMTP
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(from_email, app_password)
        server.send_message(msg)
        print("📧 Email sent successfully via Gmail.")
